In [ ]:
#PSO

import numpy as np

class Particle:
    def __init__(self, position, velocity):
        self.position = position
        self.velocity = velocity
        self.best_pos = position
        self.best_fitness = float('inf')

def pso(max_iter, N, minx, maxx, w, c1, c2):
    swarm = [Particle(np.random.uniform(minx, maxx), np.random.uniform(minx, maxx)) for _ in range(N)]
    best_pos_swarm = np.inf
    best_fitness_swarm = np.inf

    for Iter in range(max_iter):
        for i in range(N):
            r1, r2 = np.random.rand(), np.random.rand()

            # Update velocity
            swarm[i].velocity = (w * swarm[i].velocity +
                                 r1 * c1 * (swarm[i].best_pos - swarm[i].position) +
                                 r2 * c2 * (best_pos_swarm - swarm[i].position))

            # Update position
            swarm[i].position += swarm[i].velocity

            # Clip position
            if swarm[i].position < minx:
                swarm[i].position = minx
            elif swarm[i].position > maxx:
                swarm[i].position = maxx

            # Update best position and best swarm position
            swarm[i].fitness = objective_function(swarm[i].position)
            if swarm[i].fitness < swarm[i].best_fitness:
                swarm[i].best_fitness = swarm[i].fitness
                swarm[i].best_pos = swarm[i].position

            if swarm[i].fitness < best_fitness_swarm:
                best_fitness_swarm = swarm[i].fitness
                best_pos_swarm = swarm[i].position

        # End for loop over particles
    # End for loop over iterations

    # Return best particle of Swarm
    best_particle = min(swarm, key=lambda x: x.best_fitness)
    return best_particle

# Example objective function (replace this with your own)
def objective_function(x):
    return x**2  # Example function: minimize x^2

# Example usage
max_iter = 100
N = 50
minx = -10
maxx = 10
w = 0.5
c1 = 0.5
c2 = 0.5

best_particle = pso(max_iter, N, minx, maxx, w, c1, c2)
print("Best position found:", best_particle.best_pos)
print("Best fitness found:", best_particle.best_fitness)


Best position found: 2.658399924406753e-17
Best fitness found: 7.06709015808583e-34


In [ ]:
#ACO

import numpy as np

class Ant:
    def __init__(self, n_cities):
        self.route = np.random.permutation(n_cities)
        self.distance = float('inf')

def ant_colony_optimization(n_ants, max_iter, n_cities, alpha, beta, rho):
    # Initialize necessary parameters and pheromone trails
    pheromone = np.ones((n_cities, n_cities))  # Initial pheromone level
    best_route = None
    best_distance = float('inf')

    for iteration in range(max_iter):
        ants = [Ant(n_cities) for _ in range(n_ants)]

        # Calculate fitness values associated with each ant (distance of the route)
        for ant in ants:
            ant.distance = calculate_distance(ant.route)

        # Find the best solution through selection methods
        for ant in ants:
            if ant.distance < best_distance:
                best_route = ant.route
                best_distance = ant.distance

        # Update pheromone trails
        pheromone *= (1 - rho)  # Evaporation
        for ant in ants:
            for i in range(n_cities - 1):
                pheromone[ant.route[i], ant.route[i+1]] += 1 / ant.distance
            pheromone[ant.route[-1], ant.route[0]] += 1 / ant.distance  # Close the loop

    return best_route, best_distance

# Example distance calculation function (replace this with your own)
def calculate_distance(route):
    # Example function: total distance of the route in a TSP problem
    distances = np.random.rand(len(route), len(route))  # Example distances between cities
    distance = sum(distances[route[i], route[i+1]] for i in range(len(route) - 1))
    distance += distances[route[-1], route[0]]  # Close the loop
    return distance

# Example usage
n_ants = 10
max_iter = 100
n_cities = 5
alpha = 1  # Pheromone factor
beta = 1  # Heuristic factor
rho = 0.1  # Evaporation rate

best_route, best_distance = ant_colony_optimization(n_ants, max_iter, n_cities, alpha, beta, rho)
print("Best route found:", best_route)
print("Best distance found:", best_distance)


Best route found: [3 4 1 0 2]
Best distance found: 0.7475700348014175


In [ ]:
#ABC

import numpy as np

# Define the objective function to be optimized
def objective_function(x):
    return sum(x**2)

# Artificial Bee Colony Algorithm
def abc_algorithm(obj_function, n_iter, n_employed, n_onlookers, dim, lb, ub):
    # Initialization
    employed_bees = np.random.uniform(lb, ub, (n_employed, dim))
    fitness = np.array([obj_function(x) for x in employed_bees])
    best_solution_idx = np.argmin(fitness)
    best_solution = employed_bees[best_solution_idx]

    # Main loop
    for iteration in range(n_iter):
        # Employed bees phase
        for i in range(n_employed):
            phi = np.random.uniform(-1, 1, dim)
            neighbour = employed_bees[i] + phi * (employed_bees[np.random.randint(0, n_employed)] - employed_bees[i])
            neighbour = np.clip(neighbour, lb, ub)
            neighbour_fitness = obj_function(neighbour)
            if neighbour_fitness < fitness[i]:
                employed_bees[i] = neighbour
                fitness[i] = neighbour_fitness

        # Onlooker bees phase
        probabilities = 1 / (1 + fitness)
        probabilities /= probabilities.sum()
        for i in range(n_onlookers):
            chosen_bee = np.random.choice(n_employed, p=probabilities)
            phi = np.random.uniform(-1, 1, dim)
            neighbour = employed_bees[chosen_bee] + phi * (employed_bees[np.random.randint(0, n_employed)] - employed_bees[chosen_bee])
            neighbour = np.clip(neighbour, lb, ub)
            neighbour_fitness = obj_function(neighbour)
            if neighbour_fitness < fitness[chosen_bee]:
                employed_bees[chosen_bee] = neighbour
                fitness[chosen_bee] = neighbour_fitness

        # Memorize the best solution
        current_best_solution_idx = np.argmin(fitness)
        if fitness[current_best_solution_idx] < obj_function(best_solution):
            best_solution = employed_bees[current_best_solution_idx]

    return best_solution

# Example usage
if __name__ == "__main__":
    # Problem dimension
    dim = 5
    # Bounds
    lb = -5
    ub = 5
    # Number of iterations
    n_iter = 100
    # Number of employed bees
    n_employed = 20
    # Number of onlooker bees
    n_onlookers = 20

    # Run ABC algorithm
    best_solution = abc_algorithm(objective_function, n_iter, n_employed, n_onlookers, dim, lb, ub)
    print("Best solution:", best_solution)
    print("Objective value:", objective_function(best_solution))


Best solution: [ 1.79799546e-06 -2.74684609e-06  1.02711338e-06  4.88581790e-06
  2.27525815e-06]
Objective value: 4.0880929182941266e-11


In [ ]:
#PSO

import numpy as np

class Particle:
    def __init__(self, position, velocity):  # Corrected method name
        self.position = position
        self.velocity = velocity
        self.best_pos = position
        self.best_fitness = -float('inf')  # Initialize best_fitness to negative infinity for maximization

def pso(max_iter, N, minx, maxx, w, c1, c2):
    swarm = [Particle(np.random.uniform(minx, maxx), np.random.uniform(minx, maxx)) for _ in range(N)]
    best_pos_swarm = -np.inf  # Initialize best_pos_swarm to negative infinity for maximization
    best_fitness_swarm = -np.inf  # Initialize best_fitness_swarm to negative infinity for maximization

    for Iter in range(max_iter):
        for i in range(N):
            r1, r2 = np.random.rand(), np.random.rand()

            # Update velocity
            swarm[i].velocity = (w * swarm[i].velocity +
                                 r1 * c1 * (swarm[i].best_pos - swarm[i].position) +
                                 r2 * c2 * (best_pos_swarm - swarm[i].position))

            # Update position
            swarm[i].position += swarm[i].velocity

            # Clip position
            if swarm[i].position < minx:
                swarm[i].position = minx
            elif swarm[i].position > maxx:
                swarm[i].position = maxx

            # Update best position and best swarm position
            swarm[i].fitness = objective_function(swarm[i].position)
            if swarm[i].fitness > swarm[i].best_fitness:  # Modify comparison for maximization
                swarm[i].best_fitness = swarm[i].fitness
                swarm[i].best_pos = swarm[i].position

            if swarm[i].fitness > best_fitness_swarm:  # Modify comparison for maximization
                best_fitness_swarm = swarm[i].fitness
                best_pos_swarm = swarm[i].position

    # Return best particle of Swarm
    best_particle = max(swarm, key=lambda x: x.best_fitness)
    return best_particle

# Example objective function (replace this with your own)
def objective_function(x):
    return -x**2  # Example function: maximize -x^2

# Example usage
max_iter = 100
N = 50
minx = -10
maxx = 10
w = 0.5
c1 = 0.5
c2 = 0.5

best_particle = pso(max_iter, N, minx, maxx, w, c1, c2)
print("Best position found:", best_particle.best_pos)
print("Best fitness found:", best_particle.best_fitness)


Best position found: 8.115679180109129e-18
Best fitness found: -6.586424855445678e-35


In [ ]:
#TSP Using ACO
import numpy as np

class AntColony:
    def __init__(self, distances, n_ants, n_best, n_iterations, decay, alpha=1, beta=2):
        """
        Args:
            distances (2D numpy.array): Square matrix of distances. Diagonal is assumed to be np.inf.
            n_ants (int): Number of ants running per iteration
            n_best (int): Number of best ants who deposit pheromone
            n_iteration (int): Number of iterations
            decay (float): Rate at which pheromone decays. The pheromone value is multiplied by decay, so 0.95 will lead to decay, 0.5 to much faster decay.
            alpha (int or float): Exponent on pheromone, higher alpha gives pheromone more weight. Default=1
            beta (int or float): Exponent on distance, higher beta give distance more weight. Default=2
        """
        self.distances  = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    def run(self):
        shortest_path = None
        shortest_path_length = np.inf
        for i in range(self.n_iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheromone(all_paths, self.n_best, shortest_path_length)
            shortest_path, shortest_path_length = self.update_best(all_paths, shortest_path, shortest_path_length)
            self.pheromone *= self.decay
        return shortest_path, shortest_path_length

    def spread_pheromone(self, all_paths, n_best, shortest_path_length):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, path_length in sorted_paths[:n_best]:
            for move in path:
                self.pheromone[move] += 1.0 / path_length

    def update_best(self, all_paths, shortest_path, shortest_path_length):
        for path, path_length in all_paths:
            if path_length < shortest_path_length:
                shortest_path_length = path_length
                shortest_path = path
        return shortest_path, shortest_path_length

    def gen_path_dist(self, path):
        total_dist = 0
        for ele in path:
            total_dist += self.distances[ele]
        return total_dist

    def gen_all_paths(self):
        all_paths = []
        for i in range(self.n_ants):
            path = self.gen_path(0)
            all_paths.append((path, self.gen_path_dist(path)))
        return all_paths

    def gen_path(self, start):
        path = []
        visited = set()
        visited.add(start)
        prev = start
        for i in range(len(self.distances) - 1):
            move = self.pick_move(self.pheromone[prev], visited)
            path.append((prev, move))
            prev = move
            visited.add(move)
        path.append((prev, start))  # Going back to where we started
        return path

    def pick_move(self, pheromone, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0
        prob = pheromone ** self.alpha
        prob /= prob.sum()
        move = np.random.choice(self.all_inds, 1, p=prob)[0]
        return move

# Example usage:
# Define the distances between cities (Euclidean distances)
cities = np.array([
    [0, 3, 4, 2],
    [3, 0, 4, 6],
    [4, 4, 0, 5],
    [2, 6, 5, 0]
])

# Create an instance of the Ant Colony Optimization algorithm
ant_colony = AntColony(distances=cities, n_ants=5, n_best=2, n_iterations=20, decay=0.95)

# Run the Ant Colony Optimization algorithm
shortest_path, shortest_path_length = ant_colony.run()

# Print the result
print("Shortest path:", shortest_path)
print("Shortest path length:", shortest_path_length)


Shortest path: [(0, 1), (1, 2), (2, 3), (3, 0)]
Shortest path length: 14


In [13]:
import numpy as np

class ABC:
    def __init__(self, distances, n_scouts, n_employed, n_onlookers, n_iterations, limit):
        self.distances = distances
        self.n_cities = len(distances)
        self.n_scouts = n_scouts
        self.n_employed = n_employed
        self.n_onlookers = n_onlookers
        self.n_iterations = n_iterations
        self.limit = limit
        self.solutions = np.zeros((self.n_employed, self.n_cities), dtype=int)
        self.values = np.zeros(self.n_employed)
        self.best_solution = None
        self.best_value = np.inf

    def run(self):
        for i in range(self.n_iterations):
            self.employed_phase()
            self.onlooker_phase()
            self.scout_phase()
        return self.best_solution, self.best_value

    def evaluate_solution(self, solution):
        total_distance = 0
        for i in range(self.n_cities - 1):
            total_distance += self.distances[solution[i], solution[i + 1]]
        total_distance += self.distances[solution[-1], solution[0]]
        return total_distance

    def employed_phase(self):
        for i in range(self.n_employed):
            j, k = np.random.choice(range(self.n_cities), size=2, replace=False)
            new_solution = self.solutions[i].copy()
            new_solution[j], new_solution[k] = new_solution[k], new_solution[j]
            new_value = self.evaluate_solution(new_solution)
            if new_value < self.values[i]:
                self.solutions[i] = new_solution
                self.values[i] = new_value
                if new_value < self.best_value:
                    self.best_solution = new_solution
                    self.best_value = new_value
            else:
                self.limit[i] += 1

    def onlooker_phase(self):
        fitness = 1 / self.values
        total_fitness = np.sum(fitness)
        if total_fitness == 0:
            probabilities = np.ones(self.n_employed) / self.n_employed
        else:
            probabilities = fitness / total_fitness
            probabilities[np.isnan(probabilities)] = 1 / self.n_employed  # Replace NaN values with uniform probabilities
            probabilities /= np.sum(probabilities)  # Ensure probabilities sum up to 1
        for i in range(self.n_employed):
            selected = np.random.choice(range(self.n_employed), p=probabilities)
            j, k = np.random.choice(range(self.n_cities), size=2, replace=False)
            new_solution = self.solutions[selected].copy()
            new_solution[j], new_solution[k] = new_solution[k], new_solution[j]
            new_value = self.evaluate_solution(new_solution)
            if new_value < self.values[selected]:
                self.solutions[selected] = new_solution
                self.values[selected] = new_value
                if new_value < self.best_value:
                    self.best_solution = new_solution
                    self.best_value = new_value
            else:
                self.limit[selected] += 1

    def scout_phase(self):
        for i in range(self.n_employed):
            if self.limit[i] >= self.n_employed:
                self.solutions[i] = np.random.permutation(self.n_cities)
                self.values[i] = self.evaluate_solution(self.solutions[i])
                self.limit[i] = 0

# Example usage:
# Define the distances between cities (Euclidean distances)
cities = np.array([
    [0, 3, 4, 2],
    [3, 0, 4, 6],
    [4, 4, 0, 5],
    [2, 6, 5, 0]
])

# Create an instance of the Artificial Bee Colony algorithm
abc = ABC(distances=cities, n_scouts=5, n_employed=10, n_onlookers=10, n_iterations=100, limit=np.zeros(10))

# Run the Artificial Bee Colony algorithm
best_solution, best_value = abc.run()

# Print the result
print("Best solution:", best_solution)
print("Best value:", best_value)


Best solution: [2 3 0 1]
Best value: 14


<ipython-input-13-de6ee5c4b0ba>:47: RuntimeWarning: divide by zero encountered in divide
  fitness = 1 / self.values
<ipython-input-13-de6ee5c4b0ba>:52: RuntimeWarning: invalid value encountered in divide
  probabilities = fitness / total_fitness
